In [23]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/scratch/anissa.el/macro_micro_niches/macro_micro_niches2022/TMENS_analysis/notebooks/keren_building_blocks_analysis'

In [24]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.optimize import minimize,LinearConstraint, SR1
#from qpsolvers import solve_qp

from src.utils.archetypes import ArchetypalAnalysis
from src.CellAbundance import CellAbundance, generate_abundance_matrix, join_abundance_matrices
from src.utils.visualization import plot_scatter_pca, plot_cumulative_explained_variance, plot_PCAs, plot_CVEs

In [25]:
##########################################################
#      CONTROL PANEL       #  set your parameters here   #
##########################################################
patient_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 
               26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
N_SITE = 100
RADIUS = 25
# RADIUS = 25
#METHOD = "abs"#'gaussian' 
METHOD = 'gaussian'
CELL_TYPES = ['CD8-T', 'Other immune', 'DC / Mono', 'CD3-T', 'B', 'NK', 'Keratin-positive tumor', 'Tumor', 
              'CD4-T', 'Mesenchymal-like', 'Macrophages', 'Endothelial', 'Tregs', 'Unidentified', 'DC', 'Mono / Neu', 
              'Neutrophils']
ROOT_DATA_PATH = "../../data/cell_positions_data" #"../../../../fabio/output/cell_positions_data"
ROOT_OUTPUT_CSV_PATH = "../../output"
RANDOM_SEED = 1022

In [26]:
# Generating the sites centered on cells and cell abundance
#patient_ids = [1]
# Including cells at the border: border=False
abundance_matrix_center_cells = generate_abundance_matrix(CELL_TYPES, patient_ids, N_SITE, RADIUS,
                                                          METHOD,center_sites_cells=True, border=False,root=ROOT_DATA_PATH,
                                                          random_seed=RANDOM_SEED)

In [27]:
# Generating the sites and cell abundance
abundance_matrix = generate_abundance_matrix(CELL_TYPES, patient_ids, N_SITE, RADIUS,
                                             METHOD,center_sites_cells=False, root=ROOT_DATA_PATH,
                                             random_seed=RANDOM_SEED)

In [6]:
# generating the dataset with all the sites from the patients and the cells belonging to each site centered by cell (point one)

df = pd.DataFrame()
for ca in abundance_matrix_center_cells:
    df_ca = ca.get_site_cell_id_df()#get_site_cell_map_dataframe()
    df_ca['patient_id'] = int(ca.patient_id)
    df = df.append(df_ca)
df = df.reset_index(drop = True)
'''
df = [] #pd.DataFrame()
for ca in abundance_matrix_center_cells:
    df_ca = ca.get_site_cell_map_dataframe()
    df_ca['patient_id'] = int(ca.patient_id)
    df.append(df_ca)
df = pd.concat(df)
#df = df.reset_index(drop = True)
'''
df
#df.to_csv("/srv/mfs/hausserlab/anissa.el/ImmuneStates/data_BB/site_cells_gaussian.csv", index=False, header=True)#df.to_csv("{}/site_cells.csv".format(ROOT_OUTPUT_CSV_PATH), index=False, header=True)

,cell_id,cell_type,patient_id
0,272,Keratin-positive tumor,1
1,290,Keratin-positive tumor,1
2,292,Other immune,1
3,296,CD4-T,1
4,297,Macrophages,1
...,...,...,...
161834,4889,DC / Mono,41
161835,4890,B,41
161836,4895,Mesenchymal-like,41
161837,4901,Keratin-positive tumor,41


In [28]:
df_cellAb= pd.DataFrame()
for c in abundance_matrix_center_cells:
    ab_df = pd.DataFrame(c.abundance_matrix,columns = CELL_TYPES)
    ab_df['site_id'] = np.arange(len(ab_df))
    ab_df['patient_id'] = c.patient_id
    df_cellAb= df_cellAb.append(ab_df)
    
df_cellAb = df_cellAb.reset_index()
df_cellAb

,index,CD8-T,Other immune,DC / Mono,CD3-T,B,NK,Keratin-positive tumor,Tumor,CD4-T,Mesenchymal-like,Macrophages,Endothelial,Tregs,Unidentified,DC,Mono / Neu,Neutrophils,site_id,patient_id
0,0,0.000185,0.000396,0.000800,0.000490,0.000786,0.0,0.003756,0.000000,0.000684,0.000000,0.000512,0.000000,0.0,0.000000,0.0,0.000180,0.000000,0,1
1,1,0.000000,0.000342,0.000286,0.000228,0.000242,0.0,0.003504,0.000000,0.000000,0.000072,0.000246,0.000209,0.0,0.000123,0.0,0.000000,0.000000,1,1
2,2,0.000343,0.000645,0.000546,0.000654,0.000459,0.0,0.003497,0.000000,0.000531,0.000000,0.000638,0.000000,0.0,0.000085,0.0,0.000207,0.000000,2,1
3,3,0.001167,0.000385,0.000099,0.000419,0.001074,0.0,0.002347,0.000000,0.000532,0.000000,0.000716,0.000073,0.0,0.000000,0.0,0.000000,0.000000,3,1
4,4,0.000237,0.000638,0.000690,0.000613,0.000470,0.0,0.003396,0.000000,0.000696,0.000000,0.000652,0.000000,0.0,0.000045,0.0,0.000229,0.000000,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161834,3761,0.000000,0.000078,0.000255,0.000000,0.000230,0.0,0.004713,0.000000,0.000204,0.000292,0.000048,0.000000,0.0,0.000000,0.0,0.000100,0.000332,3761,41
161835,3762,0.000210,0.000509,0.001066,0.000561,0.000689,0.0,0.000426,0.000057,0.000654,0.000326,0.001064,0.000000,0.0,0.000000,0.0,0.000205,0.000169,3762,41
161836,3763,0.000043,0.000125,0.001256,0.000441,0.000432,0.0,0.001962,0.000191,0.000106,0.000501,0.001661,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3763,41
161837,3764,0.000173,0.000274,0.000223,0.000058,0.000131,0.0,0.005644,0.000087,0.000061,0.000580,0.000412,0.000078,0.0,0.000000,0.0,0.000000,0.000038,3764,41


In [29]:
#Abundance Matrix of randomly generated sites (NON CENTERED ON CELLS)
df_2 = pd.DataFrame() 
for ca in abundance_matrix:
    abundance_df = pd.DataFrame(ca.abundance_matrix,columns = CELL_TYPES)
    abundance_df['site_id'] = np.arange(len(abundance_df))
    abundance_df['patient_id'] = ca.patient_id
    df_2 = df_2.append(abundance_df)
    
df_2 = df_2.reset_index()
df_2.to_csv("../../../phenotypes_niches/data/abundance_matrix_gaussian.csv", index=False, header=True)

In [30]:
# Archetypal Analysis
sites, patients_ids,sites_ids,gradients = join_abundance_matrices(abundance_matrix)
pca = PCA()
pc = pca.fit_transform(sites)
pca3d = pd.DataFrame(pc[:, :3], columns = ['PC1', 'PC2', 'PC3'])
pca3d['site_id'] = df_2['site_id']
pca3d['patient_id'] = df_2['patient_id']
#pca3d.to_csv("/srv/mfs/hausserlab/anissa.el/ImmuneStates/data_BB/sites_pca_matrix_gaussian.csv", index=False, header=True)
AA_3D = ArchetypalAnalysis(n_archetypes = 4, 
                        tolerance = 0.001, 
                        max_iter = 200, 
                        random_state = 0, 
                        C = 0.0001, 
                        initialize = 'random',
                        redundancy_try = 30)
AA_3D.fit_transform(pc[:, :3])

array([[ 8.36960480e-04, -2.05139738e-04,  5.13431956e-04],
       [ 6.22451888e-04,  1.79993309e-04,  4.66145708e-04],
       [-5.45279156e-05, -1.05861330e-04,  5.32484013e-04],
       ...,
       [-1.11045895e-03, -8.26919234e-04,  1.67422287e-04],
       [-3.43905627e-03, -1.31344254e-03,  8.50245620e-05],
       [-2.79474995e-03, -9.07546280e-04, -2.04928635e-04]])

In [32]:
#open it
df_archetypes = pd.DataFrame(AA_3D.alfa.T, columns=["archetype"+str(i) for i in range(1,5)])#'archetype1', 'archetype2', 'archetype3', 'archetype4'])
df_archetypes['site_id'] = df_2['site_id']
df_archetypes['patient_id'] = df_2['patient_id']
df_archetypes
df_archetypes.to_csv("../../../phenotypes_niches/data/archetypes_matrix_gaussian.csv", index=False, header=True)


### Getting TMENs proportions of sites centered on cells

In [33]:
### Infer alfas of sites centered on cells (sites) from TMENs found in randomly generated sites (sites2)
from src.utils.equations import compute_cells_niches_weights,get_niches_cell_abund

n_comp = 3
sites2, patients_ids2,sites_ids2,gradients2 = join_abundance_matrices(abundance_matrix_center_cells)

#tmens = np.dot(AA_3D.archetypes.T, pca.components_[:n_comp,:])+np.mean(sites, axis=0)

tmens = get_niches_cell_abund(sitesCellAb = sites,pcaSites = pca,ArchObj=AA_3D,nComp=n_comp)
#print(tmens)
NB_TMENS = tmens.shape[0]

sites_alfas = compute_cells_niches_weights(nbNiches=NB_TMENS,niches=tmens,cellsSites =sites2)
#print(np.sum(sites_alfas[0:9,:],axis = 1))

In [34]:
### Convert niches weights of cells to DF

sites_archs = pd.DataFrame(sites_alfas, columns = ["arch1","arch2","arch3","arch4"])
sites_archs['patient_id'] = patients_ids2
sites_archs["site_id"] = sites_ids2[:,0]
sites_archs["cell_type_site"] = sites_ids2[:,1]
#print(sites_archs)
sites_archs["TOT_cell_dens"]= sites2.sum(axis=1)
print(sites_archs)
#sites_archs.to_csv("../../output/archetypes_sites_centered_all_cells_gaussian.csv",index=False,header=True)

           arch1     arch2     arch3     arch4  patient_id site_id  \
0       0.068431  0.068694  0.370227  0.492648           1     272   
1       0.027155  0.001333  0.354356  0.617156           1     290   
2       0.042043  0.073508  0.343502  0.540947           1     292   
3       0.074794  0.114432  0.237650  0.573123           1     296   
4       0.047357  0.078750  0.333755  0.540138           1     297   
...          ...       ...       ...       ...         ...     ...   
161834  0.031835  0.009153  0.469636  0.489375          41    4889   
161835  0.059731  0.074460  0.050650  0.815159          41    4890   
161836  0.022581  0.049371  0.188763  0.739284          41    4895   
161837  0.012983  0.021043  0.547638  0.418336          41    4901   
161838  0.017911  0.055750  0.166107  0.760232          41    4902   

                cell_type_site  TOT_cell_dens  
0       Keratin-positive tumor       0.007788  
1       Keratin-positive tumor       0.005253  
2              

In [39]:
#df_archetypes2.to_csv("/scratch/anissa.el/ImmuneStates/data_BB/archetypes_sites_centered_cells_gaussian.csv", index=False, header=True)
sites_archs.to_csv("../../output/archetypes_sites_centered_cells_gaussian2.csv",index=False,header=True)
sites_archs.to_csv("../../../phenotypes_niches/data/archetypes_sites_centered_all_cells_gaussian2.csv",index=False,header=True)

In [36]:
n_comp = 3
res = np.dot(AA_3D.archetypes.T, pca.components_[:n_comp,:])+np.mean(sites, axis=0)
archetypes_cell_space = pd.DataFrame(res,  columns = CELL_TYPES)
archetypes_cell_space["archetype"] = ['ARCH1', 'ARCH2', 'ARCH3', 'ARCH4']
archetypes_cell_space

,CD8-T,Other immune,DC / Mono,CD3-T,B,NK,Keratin-positive tumor,Tumor,CD4-T,Mesenchymal-like,Macrophages,Endothelial,Tregs,Unidentified,DC,Mono / Neu,Neutrophils,archetype
0,-0.001750,-0.000236,0.000082,-0.000005,0.013639,-0.000003,-0.000369,0.000106,0.005125,0.000288,-0.002278,0.000020,-0.000121,0.000022,0.000176,-0.000253,-0.000047,ARCH1
1,0.006750,0.000587,0.001184,0.000768,-0.001002,0.000393,0.000359,-0.000257,0.006473,-0.000456,0.003933,0.000154,0.000437,-0.000095,0.000174,0.000242,0.000211,ARCH2
2,0.000017,-0.000375,-0.000140,-0.000084,-0.000092,-0.000011,0.010285,0.000116,-0.000131,-0.000210,-0.000242,-0.000092,0.000044,-0.000040,-0.000041,0.000007,-0.000015,ARCH3
3,0.000130,0.000605,0.000248,0.000184,0.000030,-0.000003,-0.000131,0.000158,-0.000272,0.000718,0.001126,0.000180,0.000007,0.000141,0.000053,0.000183,0.000189,ARCH4


In [37]:
#TODO: set column and row names when converting dataframe to csv file!

archetypes_cell_space.to_csv("../../output/archetypes_cellAbundance.csv",encoding='utf-8',index=False,header=True)
#archetypes_cell_space.to_csv("/scratch/anissa.el/ImmuneStates/data_BB/archetypes_building_blocks_17cells_space_gaussian.csv",index=False,header=True)


In [38]:
archetypes_cell_space.columns

Index(['CD8-T', 'Other immune', 'DC / Mono', 'CD3-T', 'B', 'NK',
       'Keratin-positive tumor', 'Tumor', 'CD4-T', 'Mesenchymal-like',
       'Macrophages', 'Endothelial', 'Tregs', 'Unidentified', 'DC',
       'Mono / Neu', 'Neutrophils', 'archetype'],
      dtype='object')